*IMPORT LIBRARY*

In [ ]:
%pip install selenium

In [ ]:
%from selenium import webdriver
%from selenium.webdriver.common.keys import Keys
%from selenium.webdriver.support import expected_conditions as EC
%from selenium.webdriver.common.by import By
%from selenium.webdriver.support.wait import WebDriverWait
import os
%from webdriver_manager.chrome import ChromeDriverManager
%from selenium.webdriver.chrome.service import Service as ChromeService
%from selenium.webdriver.chrome.options import Options

*Buka Chrome dan langsung masuk ke instagram*

In [ ]:
driver = Options()
driver.add_argument("--ignore-certificate-errors")
driver.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")
driver.add_experimental_option("detach", True)
driver.add_argument("--start-maximized")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=driver)
driver.get("https://www.instagram.com/")

*INPUT USERNAME DAN PASSWORD*

In [ ]:
# Wait for the username field to be clickable
username = WebDriverWait(driver, 10).until(
EC.element_to_be_clickable((By.NAME, "username"))
)
username.clear()
username.send_keys("sentis2872")

# Wait for the password field to be clickable
password = WebDriverWait(driver, 10).until(
EC.element_to_be_clickable((By.NAME, "password"))
)
password.clear()
password.send_keys("SENTISprigel")

# Click the login button
login_button = WebDriverWait(driver, 10).until(
EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
)
login_button.click()


*KLIK BUTTON LOGIN*

In [ ]:
log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

*KLIK BUTTON NOT NOW*

In [ ]:
not_now_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[text()='Not Now']")))
not_now_button.click()

*KLIK BUTTON NOT NOW*

In [ ]:
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not Now')]"))).click()

*MENCARI SEARCHBOX*

In [ ]:
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[text()='Search']"))).click()

*KETIK #UNNES DI SEARCHBOX*

In [ ]:
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()
keyword = "pesanunnes"
searchbox.send_keys(keyword)

*KLIK #UNNES DI PENCARIAN PERTAMA*

In [ ]:
element = driver.find_element(By.XPATH, "//span[contains(text(), 'pesanunnes')]").click()

*DAPATIN LINK INSTAGRAMNYA*

In [ ]:
#target all the link elements on the page
anchors = driver.find_elements('tag name', 'a')
anchors = [a.get_attribute('href') for a in anchors]
#narrow down all links to image links only
anchors = [a for a in anchors if str(a).startswith("https://www.instagram.com/p/")][:5]

print('Found ' + str(len(anchors)) + ' links to images')
anchors

*SCROLL DOWN, EKSTRAK CAPTION, DAN SAVE DI CSV*

In [ ]:
comments = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj"))
            )
for comment in comments:
                # Periksa apakah elemen span memiliki anak elemen <a>
    if not comment.find_elements(By.TAG_NAME, 'a'):
        comment_text = comment.text
        print(f"Banyaknya: {len(comments)}")
        print(f"Comment: {comment_text}")

In [ ]:
usernames = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span._ap3a._aaco._aacw._aacx._aad7._aade")))
for username in usernames:
    username_text = username.text
    if 'pesanunnes' not in username_text:
        print(f"Username: @{username_text}")

In [ ]:
import csv
import time

SCROLL DOWN

In [ ]:
scrollable_div = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6"))
            )
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)

In [ ]:
with open('comments_usernames.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL', 'Username', 'Comment'])

    # Loop melalui setiap URL postingan (hanya 3)
    for link in anchors:
        # Buka URL postingan
        driver.get(link)
        print(f"Visiting {link}")
        time.sleep(5)  # Tambahkan waktu tunggu untuk memastikan halaman telah dimuat

        try:
            # Temukan elemen yang dapat digulir
            scrollable_div = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6"))
            )
            
            last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            new_comments_loaded = True
            scroll_attempts = 0

            while new_comments_loaded and scroll_attempts < 10:  # Batasi maksimal 10 kali scroll
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
                time.sleep(5)  # Tambahkan waktu tunggu untuk memuat komentar
                new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
                if new_height == last_height:
                    new_comments_loaded = False
                else:
                    last_height = new_height
                    scroll_attempts += 1

            # Tambahkan waktu tunggu setelah semua scroll selesai untuk memastikan komentar dimuat
            time.sleep(5)

            # Scroll kembali ke atas
            driver.execute_script("arguments[0].scrollTop = 0", scrollable_div)
            time.sleep(2)  # Waktu tunggu setelah scroll ke atas
            
            # Tunggu hingga elemen komentar muncul
            comments = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj"))
            )
            usernames = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span._ap3a._aaco._aacw._aacx._aad7._aade")))

            for username, comment in zip(usernames, comments):
                # Periksa apakah elemen span memiliki anak elemen <a>
                if not comment.find_elements(By.TAG_NAME, 'a'):
                    username_text = username.text
                    comment_text = comment.text
                    print(f"Username: @{username_text}, Comment: {comment_text}")

                    # Tulis URL postingan, username, dan komentar ke dalam file CSV
                    writer.writerow([link, f"@{username_text}", comment_text])
                    print(f"Wrote to CSV: {link}, @{username_text}, {comment_text}")

        except Exception as e:
            # Jika terjadi kesalahan
            print("Error:", e)
            print("Could not retrieve data for:", link)


In [ ]:
with open('comments_usernames.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL', 'Username', 'Comment'])

    # Loop melalui setiap URL postingan (hanya 3)
    for link in anchors:
        # Buka URL postingan
        driver.get(link)  
        try:
            comments = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj"))
            )
            usernames = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span._ap3a._aaco._aacw._aacx._aad7._aade")))
            # for username in usernames:
            #     username_text = username.text
            #     if 'pesanunnes' not in username_text:
            #         print(f"Username: @{username_text}")
            
            for username, comment in zip(usernames, comments):
                # Periksa apakah elemen span memiliki anak elemen <a>
                if not comment.find_elements(By.TAG_NAME, 'a'):
                    username_text = username.text
                    comment_text = comment.text
                    print(f"Username: @{username_text}, Comment: {comment_text}")
                    writer.writerow([link, f"@{username_text}", comment_text])
                    print(f"Wrote to CSV: {link}, @{username_text}, {comment_text}")
            
            # Tulis URL postingan dan caption ke dalam file CSV
            

        except Exception as e:
            # Jika terjadi kesalahan
            print("Error:", e)
            print("Could not retrieve data for:", link)

*INI CAPTION*

In [ ]:
# Buka file CSV untuk menulis
with open('captions1.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['Post URL', 'Username', 'Comment'])

    # Loop melalui setiap URL postingan
    for link in anchors:
        # Buka URL postingan
        driver.get(link)
        
        try:
            # Tunggu hingga elemen span muncul
            span_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj")))
            
            # Ambil teks dari elemen span
            caption_text = span_element.text
            
            # Tulis URL postingan dan caption ke dalam file CSV
            writer.writerow([link, caption_text])
        
        except Exception as e:
            # Jika tidak ada elemen span atau terjadi kesalahan lainnya
            print("Error:", e)
            print("No caption found for:", link)